<a href="https://colab.research.google.com/github/ludmillalopes/AgentesIA_passos_de_ballet/blob/main/Passos_de_Ballet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
!pip install -q google-genai

In [2]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [24]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

from IPython.display import Image, display

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [52]:
##########################################
# --- Agente 1: Verifica calendário Google --- #
##########################################
def agente_buscador(passo_danca):
  buscador = Agent(
      name='agente_buscador',
      model='gemini-2.0-flash',
      description='Agente que busca o passo de dança',
      instruction='''
      Você é professor de dança, especificamente Ballet Clássico.
      A sua tarefa é buscar o passo de dança que foi solicitado.
      Caso não encontre da forma exata a qual foi escrita, procure por um passo em que a escrita seja semelhante.
      Lembrando que normalmente os nomes dos passos são escritos em francês.
      Você pode usar o modo como o nome soa vocalmente em francês e assimilar à escrita em português,
      para ajudar os usuários que não sabem qual é a correta ortografia da palavra.
      Apresente o nome do passo que foi encontrado. Se for encontrado mais de um, liste o nome de todos eles.
      Adicione uma brevíssima descrição que seja suficiente para que o usuário consiga identifica-lo e
      ''',
      # uma imagem que ilustra o passo.
      # pesquise na ferramenta de busca do Google (google-search) uma imagem que ilustra o passo.
      # tools=[google_search]
  )
  # Executa o agente
  passos_encontrados = call_agent(buscador, passo_danca)
  return passos_encontrados

In [96]:
##########################################
# --- Agente 2: Estuda o passo de dança --- #
##########################################
def agente_estudante(passo_danca):
  estudante = Agent(
      name='agente_estudante',
      model='gemini-2.0-flash',
      description='Agente que estuda o passo de dança solicitado',
      instruction='''
      Você é professor de dança, especificamente Ballet Clássico.
      A sua tarefa é explicar didaticamente, mas de modo sucinto e direto, o passo de dança que foi solicitado.
      Apresente as seguinte informações:
      - Nome do passo;
      - Explicação de como o nome do passo é pronunciado corretamente;
      - Qual o tipo de passo (salto, giro, pose, passo de transição, etc)
      - Em qual momento da aula o passo costuma ser ensinado (por exemplo: na barra, no centro, allegro, etc);
      - Explicação sobre o que é o passo;
      - Indique se existir diferença na execução em diferentes vertentes do Ballet Clássico;
      - Como é executado;
      - Possíveis erros de execução;
      - Dicas de exercício para chegar na execução perfeita.
      -----

      A estética do texto pode ser formatado de forma mais divertida, com emojis e mais colorido.
      ''',
  )
  # Executa o agente
  passos_sobre = call_agent(estudante, passo_danca)
  return passos_sobre

In [97]:
# --- Obter o Tópico do Usuário ---
passo_danca = input('Olá, digite o nome do passo de ballet que quer entender: ')

# Inserir lógica do sistema de agentes ################################################
if not passo_danca:
  print('Você esqueceu de digitar o nome do passo de ballet. 😿')
else:
  passos_buscados = agente_buscador(passo_danca)
  display(to_markdown(passos_buscados))

  # Confirma se está certo o passo de dança
  confirma_passo = input('✔️ O passo de ballet foi listado? Digite novamente para confirmar! 😄 ')

  if not confirma_passo:
    print('Você esqueceu de digitar o passo de dança! Vamos começar novamente? 🙃')
  else:
    passo_tudo_sobre = agente_estudante(confirma_passo)
    display(to_markdown(passo_tudo_sobre))


Olá, digite o nome do passo de ballet que quer entender: plie


> - Plié (dobrado): Flexão dos joelhos com rotação externa dos membros inferiores. Pode ser demi-plié (flexão
>  parcial) ou grand-plié (flexão completa).


✔️ O passo de ballet foi listado? Digite novamente para confirmar! 😄 Plié


> ✨ Vamos aprender sobre o Plié! ✨
> 
> 💃 **Nome do Passo:** Plié (pronuncia-se "pliê")
> 
> 🤸 **Tipo de Passo:** É um passo fundamental de flexão dos joelhos. Serve como base para quase todos os movimentos no ballet!
> 
> 📍 **Onde é Ensinado:** Geralmente, o plié é um dos primeiros exercícios ensinados na barra e no centro, desde as primeiras aulas!
> 
> 🤔 **O Que É:** O plié é uma flexão dos joelhos que alonga e fortalece os músculos das pernas e quadris, além de melhorar o equilíbrio e a flexibilidade. É a base de quase todos os movimentos do ballet!
> 
> 🩰 **Variações:**
>    *   **Demi-plié:** Flexão parcial dos joelhos, sem tirar os calcanhares do chão (a não ser na segunda posição).
>    *   **Grand plié:** Flexão completa dos joelhos, com os calcanhares saindo do chão (exceto na segunda posição).
> 
> ⚙️ **Como Executar:**
> 
> 1.  Comece em uma das cinco posições básicas do ballet.
> 2.  Mantenha os calcanhares no chão (no demi-plié) ou levante-os gradualmente (no grand plié, exceto na segunda posição).
> 3.  Flexione os joelhos para fora, mantendo os quadris alinhados e o corpo ereto.
> 4.  Retorne à posição inicial esticando os joelhos e abaixando os calcanhares (no grand plié).
> 
> ⚠️ **Erros Comuns:**
> 
> *   ❌ Inclinar o corpo para frente ou para trás.
> *   ❌ Deixar os joelhos caírem para dentro.
> *   ❌ Levantar os ombros ou tensionar o pescoço.
> *   ❌ Não manter o alinhamento do quadril.
> 
> 💪 **Dicas e Exercícios:**
> 
> *   🌟 **Alongamento:** Alongue os músculos da coxa e panturrilha regularmente.
> *   🌟 **Fortalecimento:** Faça exercícios de fortalecimento para os músculos das pernas e do core.
> *   🌟 **Prática:** Pratique o plié diariamente, concentrando-se na forma correta e no alinhamento do corpo. Use um espelho para verificar sua postura!
> 
> 🌈 **Lembre-se:** O plié é a base de tudo no ballet. Com prática e atenção, você vai dominar esse passo essencial!
> 
